## Project two: character-level language modeling in TensorFlow

### Preprocessing the dataset

In [1]:
! curl -O https://www.gutenberg.org/files/1268/1268-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1143k  100 1143k    0     0   131k      0  0:00:08  0:00:08 --:--:--  209k


In [2]:
import numpy as np

## Reading and preprocessing text
with open('1268-0.txt', 'r') as fp:
    text = fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))
print('Unique Characters:', len(char_set))

Total Length: 1130711
Unique Characters: 85


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)
print('Text encoded shape:', text_encoded.shape)

print(text[:15], '== Encoding ==>', text_encoded[:15])
print(text_encoded[:15], '== Reverse ==>', ''.join(char_array[text_encoded[:15]]))

Text encoded shape: (1130711,)
THE MYSTERIOUS  == Encoding ==> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]
[48 36 33  1 41 53 47 48 33 46 37 43 49 47  1] == Reverse ==> THE MYSTERIOUS 


Now, we will create a TensorFlow dataset from this array:

In [4]:
import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

2023-08-18 10:33:49.621790: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 10:33:49.701904: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 10:33:49.703011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 10:33:51.470447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


48 -> T
36 -> H
33 -> E
1 ->  
41 -> M


2023-08-18 10:33:53.433634: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
seq_length = 40
chunk_size = 41

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

In [6]:
## define the function for splitting x & y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [7]:
## inspection:
for example in ds_sequences.take(2):
    print(' Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

 Input (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTER'
Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERI'

 Input (x): 'OUS ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPAR'
Target (y): 'US ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPART'



In [8]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)# drop_remainder=True)

ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))>

### Building a character-level RNN model